In [2]:
import sys
from antlr4 import *
from parser.CodeGen.CodeGenLexer import CodeGenLexer
from parser.CodeGen.CodeGenParser import CodeGenParser

# Custom visitor to extract the structure from the parsed input
class CodeGenVisitor(ParseTreeVisitor):

    def visitCodeSpec(self, ctx):
        function_name = ctx.functionName().getText()
        parameters = [param.getText() for param in ctx.paramList().param()]
        purpose = ctx.functionPurpose().getText().strip('"')
        return {
            'function_name': function_name,
            'parameters': parameters,
            'purpose': purpose
        }

# Function to parse the input
def parse_code_spec(input_text):
    input_stream = InputStream(input_text)
    lexer = CodeGenLexer(input_stream)
    stream = CommonTokenStream(lexer)
    parser = CodeGenParser(stream)
    tree = parser.codeSpec()
    visitor = CodeGenVisitor()
    return visitor.visit(tree)

# Example usage
description = 'Create a function calculateSum with parameters a, b that "returns the sum of two numbers"'
parsed_data = parse_code_spec(description)
print(parsed_data)

{'function_name': 'calculateSum', 'parameters': ['a', 'b'], 'purpose': 'returns the sum of two numbers'}


In [3]:
from openai import OpenAI

with open("/home/loc/Documents/keys/OPENAI_API_KEY.txt") as f:
    OPENAI_API_KEY = f.read().strip()
    
client = OpenAI(api_key=OPENAI_API_KEY)

# Function to prompt the LLM
def generate_code_with_llm(parsed_data):
    prompt = (
        f"Generate a Python function named '{parsed_data['function_name']}' "
        f"with parameters {', '.join(parsed_data['parameters'])}. "
        f"The function should {parsed_data['purpose']}."
    )
    
    response = client.chat.completions.create(
                messages=[{
                    "role": "user",
                    "content": prompt,
                }],
                model="gpt-4o-mini",
            )

    
    return response.choices[0].message.content

code = generate_code_with_llm(parsed_data)
print(code)

You can create a simple Python function named `calculateSum` that takes two parameters, `a` and `b`, and returns their sum. Here's the code:

```python
def calculateSum(a, b):
    return a + b
```

You can use this function like this:

```python
result = calculateSum(5, 7)
print(result)  # Output: 12
``` 

This function works for both integers and floating-point numbers.
